In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time
import math


X = tf.placeholder(tf.float32, [None, 784])


init = tf.global_variables_initializer()
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True) 


Y_ = tf.placeholder(tf.float32, [None, 10])

lr = tf.placeholder(tf.float32)

pkeep = tf.placeholder(tf.float32)

L = 200
M = 100
N = 50
O = 25

W1 = tf.Variable(tf.truncated_normal([784, L], stddev=0.1))  
B1 = tf.Variable(tf.zeros([L]))
W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.zeros([M]))
W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
B3 = tf.Variable(tf.zeros([N]))
W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
B4 = tf.Variable(tf.zeros([O]))
W5 = tf.Variable(tf.truncated_normal([O, 10], stddev=0.1))
B5 = tf.Variable(tf.zeros([10]))

Y1 = tf.nn.relu(tf.matmul(X, W1) + B1)
Y1d = tf.nn.dropout(Y1, pkeep)
Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
Y2d = tf.nn.dropout(Y2, pkeep)
Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
Y3d = tf.nn.dropout(Y3, pkeep)
Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
Y4d = tf.nn.dropout(Y4, pkeep)
Ylogits = tf.matmul(Y4d, W5) + B5
Y = tf.nn.softmax(Ylogits)

cost = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
cost = tf.reduce_mean(cost)*100



optimizer = tf.train.AdamOptimizer(0.001)
train_step = optimizer.minimize(cost)



correct_prediction = tf.equal(tf.arg_max(Y, 1), tf.arg_max(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)




for i in range(100000):
    # load batch of images and correct answers
    batch_X, batch_Y = mnist.train.next_batch(100)
   
   
    train_data={X: batch_X, Y_: batch_Y,pkeep : [0.75], lr: 0.001}
    # train
    sess.run(train_step, feed_dict=train_data)

a,c = sess.run([accuracy, cost], feed_dict=train_data)

test_data={X: mnist.test.images, Y_: mnist.test.labels,pkeep : [1] }
a,c = sess.run([accuracy, cost], feed_dict=test_data)
print('Accuracy {0}'.format(a))
    

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Accuracy 0.98290002346
